In [1]:
# get current directory as idlpy will change it
import os

cwd = os.getcwd()

from idlpy import *
import pandas as pd
import xarray as xr

# restore current directory
os.chdir(cwd)

IDL 8.8.3 (darwin x86_64 m64).
(c) 2022, L3Harris Geospatial Solutions, Inc.

A new version is available: IDL 8.9
https://www.l3harrisgeospatial.com/Product-Downloads/
 
Licensed for use by: UCLA - Dept of Geophysics
License: 222541


In [2]:
event01 = {
    "probe": "c",
    "tstart": "2020-02-19/06:27:00",
    "tstop": "2020-02-19/06:28:00",
    "datatype": "fgl",
}


event02 = {
    "probe": "c",
    "tstart": "2021-04-03/04:57:30",
    "tstop": "2021-04-03/04:58:30",
    "datatype": "fgl",
}

event03 = {
    "probe": "c",
    "tstart": "2022-04-04/18:58:00",
    "tstop": "2022-04-04/18:59:00",
    "datatype": "fgl",
}

In [3]:
ds = [event01, event02, event03]
pd.DataFrame(ds)

,probe,tstart,tstop,datatype
0,c,2020-02-19/06:27:00,2020-02-19/06:28:00,fgl
1,c,2021-04-03/04:57:30,2021-04-03/04:58:30,fgl
2,c,2022-04-04/18:58:00,2022-04-04/18:59:00,fgl


In [23]:
event = event02

In [42]:
probe = event["probe"]

idl_script = f"""
    probe = '{probe}'
    datatype = 'peir'

    tstart = '{event["tstart"]}'
    tstop = '{event["tstop"]}'
    trange = [tstart, tstop]

    thm_part_load, probe=probe, trange=trange, datatype=datatype
    thm_part_products, probe=probe, datatype=datatype, trange=trange, units='df'
    get_data, 'th{probe}_peir_df_energy', data=th{probe}_peir_df_energy
    tplot, 'th{probe}_peir_df_energy'
"""

for line in idl_script.splitlines():
    IDL.run(line)
    # print(line)

d = IDL.thc_peir_df_energy

In [44]:
event

{'name': 'event02',
 'probe': 'c',
 'datatype': 'fgl',
 'tstart': '2021-04-03/04:57:30',
 'tstop': '2021-04-03/04:58:30'}

In [52]:
d = IDL.thc_peir_df_energy
time = pd.to_datetime(d["X"], unit="s")
energy = xr.DataArray(d["V"], dims=["energy"], name="energy", attrs={"units": "eV"})
df = xr.DataArray(
    d["Y"], 
    dims=["energy", "time"], 
    coords={"time": time, "energy": energy},
    name=f'th{probe}_peir_df_energy',
    attrs={"units": "s^3/cm^3/km^3", "long_name": "PSD"},
)
df.to_netcdf(f'./data/{event["name"]}_df.nc')

In [ ]:
# probe='c'
# datatype='peir'

# tstart='2020-02-19/06:27:00'
# tstop='2020-02-19/06:28:00'
# trange = [tstart, tstop]

# thm_part_load,probe=probe,trange=trange,datatype=datatype
# thm_part_products,probe=probe,datatype=datatype,trange=trange, units='df'
# ; thm_load_fgm, probe=probe, trange=trange, level='l2', coord='gse', /time_clip
# ; copy the data to a new variable
# ; copy_data, 'th'+probe+'_'+datatype+'_gse', 'thx_fgm'#

# ; makes a single transformation matrix that covers the entire interval
# tvar='thx_fgm'
# tvar_lmn = 'thx_fgm_lmn'
# minvar_matrix_make, 'thx_fgm', tstart=tstart, tstop=tstop
# tvector_rotate, 'thx_fgm_mva_mat', 'thx_fgm', newname=tvar_lmn
# ; update labels and coordinate system
# get_data, tvar_lmn, data=d, dlimits=dl, limits=l
# dl.data_att.coord_sys = 'lmn'
# dl.labels = ['bl', 'bm', 'bn']
# dl.ytitle = tvar_lmn
# dl.ysubtitle = '[nT LMN]'
# store_data, tvar_lmn, data=d, dlimits=dl, limits=l

# tplot, [tvar, tvar_lmn]
# makepng, 'thx_fgm_lmn '

# ; code-summary: plot bl and bm component
# get_data, 'thx_fgm', data=thx_fgm
# get_data, 'mva_data', data=mva_data
# help, thx_fgm.y, mva_data.y
# plot(thx_fgm.y[*,0],thx_fgm.y[*,1])
# plot(mva_data.y[*,0],mva_data.y[*,1])